In [7]:
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import math
from math import hypot
import seaborn as sns
import xgboost as xgb
from xgboost import XGBClassifier

# NFL 2021 Big Data Bowl

## Get Data

In [28]:
plays = pd.read_csv(r"/Users/ishanidesai/Documents/sports_analytics/nfl-big-data-bowl-2021/plays.csv")
week1 = pd.read_csv(r"/Users/ishanidesai/Documents/sports_analytics/nfl-big-data-bowl-2021/week1.csv")
week2 = pd.read_csv(r"/Users/ishanidesai/Documents/sports_analytics/nfl-big-data-bowl-2021/week2.csv")
week3 = pd.read_csv(r"/Users/ishanidesai/Documents/sports_analytics/nfl-big-data-bowl-2021/week3.csv")
week4 = pd.read_csv(r"/Users/ishanidesai/Documents/sports_analytics/nfl-big-data-bowl-2021/week4.csv")
week5 = pd.read_csv(r"/Users/ishanidesai/Documents/sports_analytics/nfl-big-data-bowl-2021/week5.csv")
week6 = pd.read_csv(r"/Users/ishanidesai/Documents/sports_analytics/nfl-big-data-bowl-2021/week6.csv")
week7 = pd.read_csv(r"/Users/ishanidesai/Documents/sports_analytics/nfl-big-data-bowl-2021/week7.csv")
week8 = pd.read_csv(r"/Users/ishanidesai/Documents/sports_analytics/nfl-big-data-bowl-2021/week8.csv")

first_8_weeks = pd.concat([week1, week2, week3, week4, week5, week6, week7, week8])

# JOINING WEEK 1 TRACKING DATA AND PLAY DATA
play_tracking = plays.merge(first_8_weeks, how = 'inner', on=['gameId', 'playId'], sort=False)

cols = ['gameId', 'playId', 'quarter', 'down', 'yardsToGo', 'playType', 'defendersInTheBox', 'numberOfPassRushers', 'personnelD', 'isDefensivePI', 'passResult', 'time', 'x', 'y', 's', 'a', 'dis', 'event', 'nflId', 'displayName', 'jerseyNumber', 'position', 'frameId', 'team', 'playDirection', 'route']
play_tracking = play_tracking[cols]


## Combining Tracking Data w/ Data From First 8 Weeks

In [30]:
play_tracking = play_tracking.loc[play_tracking['playType'] == 'play_type_pass']

eight_weeks = first_8_weeks['gameId'].unique()
print(eight_weeks)
play_info = plays[plays.gameId.isin(eight_weeks)]
play_info['off_dist'] = -1
play_info['def_dist'] = -1
play_info['defender_receiver_dist'] = -1
play_info['closest_defender'] = ''
# only want plays where pass result is interception or incomplete, and only the frames where the action int/incomplete happens
play_info = play_info[play_info.passResult.isin(['IN', 'I', 'C'])]

play_tracking = play_tracking[play_tracking.event.isin(['pass_outcome_incomplete', 'pass_outcome_interception', 'pass_outcome_caught'])]

print(play_tracking.shape)
play_tracking.head()


[2018090600 2018090906 2018090907 2018090903 2018090901 2018090902
 2018090900 2018090905 2018090909 2018090910 2018090912 2018091000
 2018091001 2018091300 2018091606 2018091605 2018091603 2018091602
 2018091607 2018091608 2018091600 2018091601 2018091604 2018091610
 2018091609 2018091612 2018091611 2018091613 2018091700 2018092000
 2018092306 2018092309 2018092304 2018092307 2018092302 2018092305
 2018092308 2018092301 2018092300 2018092303 2018092310 2018092311
 2018092312 2018092313 2018092400 2018092700 2018093003 2018093001
 2018093007 2018093006 2018093000 2018093004 2018093005 2018093002
 2018093009 2018093008 2018093010 2018093011 2018093012 2018100100
 2018100400 2018100700 2018100704 2018100706 2018100707 2018100703
 2018100701 2018100705 2018100702 2018100708 2018100709 2018100710
 2018100711 2018100712 2018100800 2018101100 2018101406 2018101405
 2018101401 2018101404 2018101402 2018101400 2018101403 2018101408
 2018101407 2018101409 2018101411 2018101410 2018101412 201810

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

(119285, 26)


,gameId,playId,quarter,down,yardsToGo,playType,defendersInTheBox,numberOfPassRushers,personnelD,isDefensivePI,passResult,time,x,y,s,a,dis,event,nflId,displayName,jerseyNumber,position,frameId,team,playDirection,route
714,2018090600,75,1,1,15,play_type_pass,7.0,4.0,"4 DL, 2 LB, 5 DB",False,C,2018-09-07T01:07:19.700Z,96.10,27.82,0.99,1.14,0.10,pass_outcome_caught,310.0,Matt Ryan,2.0,QB,52,away,left,NaN
715,2018090600,75,1,1,15,play_type_pass,7.0,4.0,"4 DL, 2 LB, 5 DB",False,C,2018-09-07T01:07:19.700Z,81.74,49.08,7.94,2.37,0.80,pass_outcome_caught,79848.0,Malcolm Jenkins,27.0,SS,52,home,left,NaN
716,2018090600,75,1,1,15,play_type_pass,7.0,4.0,"4 DL, 2 LB, 5 DB",False,C,2018-09-07T01:07:19.700Z,81.48,50.13,4.93,3.07,0.49,pass_outcome_caught,2495454.0,Julio Jones,11.0,WR,52,away,left,HITCH
717,2018090600,75,1,1,15,play_type_pass,7.0,4.0,"4 DL, 2 LB, 5 DB",False,C,2018-09-07T01:07:19.700Z,81.41,19.08,4.65,2.84,0.46,pass_outcome_caught,2495613.0,Corey Graham,24.0,FS,52,home,left,NaN
718,2018090600,75,1,1,15,play_type_pass,7.0,4.0,"4 DL, 2 LB, 5 DB",False,C,2018-09-07T01:07:19.700Z,76.75,36.60,2.82,1.85,0.28,pass_outcome_caught,2533040.0,Mohamed Sanu,12.0,WR,52,away,left,HITCH


## Calculate Distances between receiver/defenders, and from the ball

In [31]:
dists = pd.DataFrame(columns=['playId', 'distance'])
# for every play (playid), get the football x and y. find the distance between the football xy
# get closest for home and closest for away. get distance between those 2and each player's xy
not_counted = 0
print(play_tracking.event.unique())

for gameId in play_tracking.gameId.unique():
    game_specific = play_tracking.loc[play_tracking['gameId'] == gameId]
    
    for playId in game_specific.playId.unique():
        # get the tracking data for players/ball in specific play
        play_specific = game_specific.loc[game_specific['playId'] == playId]
        
        # get x, y coords of football
        football = play_specific.loc[play_specific['displayName'] == 'Football']
        if (len(football) == 1):
            football_x = football['x'].iloc[0]
            football_y = football['y'].iloc[0]
        else:
            #print(play_specific)
            not_counted += 1
            continue
        
        players = play_specific.loc[play_specific['displayName'] != 'Football']
        
        # figure out whether home or away team is on offense based on QB
        qb_row = play_specific.loc[play_specific['position'] == 'QB']
        if (len(qb_row == 1)):
            offense = qb_row['team'].iloc[0]
        else:
            #print(play_specific)
            not_counted += 1
            continue
            
        first_off = 1 # flags for initial offense and defense values 
        first_def = 1
        off_dist = 0  # distance from the ball of closest receiver and defender
        def_dist = 0
        off_closest = 0 # rows w/ player info of closest receiver and defender
        def_closest = 0
        
        # for every player in the play, calculate their distance from the football check if its the closest
        for index, row in players.iterrows():
            player_x = row['x']
            player_y = row['y']
            dist = math.hypot(football_x - player_x, football_y - player_y)
            
            if ((first_off == 1) and (row['team'] == offense)):
                first_off = 0
                off_closest = row
                off_dist = dist
                continue
            elif ((first_def == 1) and (row['team'] != offense)):
                first_def = 0
                def_closest = row
                def_dist = dist
                continue

            if ((row['team'] == offense) and (dist < off_dist)):
                off_closest = row
                off_dist = dist
            elif ((row['team'] != offense) and (dist < def_dist)):
                def_closest = row
                def_dist = dist
                
        if ((type(def_closest) != int) and type(off_closest) != int):        
            play_info.loc[(play_info.playId == playId) & (play_info.gameId == gameId), 'off_dist'] = off_dist
            play_info.loc[(play_info.playId == playId) & (play_info.gameId == gameId), 'def_dist'] = def_dist

            dist = math.hypot(off_closest['x'] - def_closest['x'], off_closest['y'] - def_closest['y'])
            play_info.loc[(play_info.playId == playId) & (play_info.gameId == gameId), 'defender_receiver_dist'] = dist
            play_info.loc[(play_info.playId == playId) & (play_info.gameId == gameId), 'closest_defender'] = def_closest['displayName']
            #print(gameId, playId, off_dist)
print(play_info.shape)
print(not_counted)
play_info.head()


['pass_outcome_caught' 'pass_outcome_incomplete'
 'pass_outcome_interception']
(8662, 31)
22


,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,playType,yardlineSide,yardlineNumber,offenseFormation,personnelO,defendersInTheBox,numberOfPassRushers,personnelD,typeDropback,preSnapVisitorScore,preSnapHomeScore,gameClock,absoluteYardlineNumber,penaltyCodes,penaltyJerseyNumbers,passResult,offensePlayResult,playResult,epa,isDefensivePI,off_dist,def_dist,defender_receiver_dist,closest_defender
0,2018090600,75,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,ATL,play_type_pass,ATL,20,I_FORM,"2 RB, 1 TE, 2 WR",7.0,4.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0.0,0.0,15:00:00,90.0,NaN,NaN,C,10,10,0.261827,False,0.751066,1.813395,1.081712,Malcolm Jenkins
1,2018090600,146,(13:10) M.Ryan pass incomplete short right to ...,1,1,10,ATL,play_type_pass,PHI,39,SINGLEBACK,"1 RB, 1 TE, 3 WR",7.0,4.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0.0,0.0,13:10:00,49.0,NaN,NaN,I,0,0,-0.372360,False,2.051975,3.202062,4.924896,Jordan Hicks
2,2018090600,168,(13:05) (Shotgun) M.Ryan pass incomplete short...,1,2,10,ATL,play_type_pass,PHI,39,SHOTGUN,"2 RB, 1 TE, 2 WR",6.0,4.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0.0,0.0,13:05:00,49.0,NaN,NaN,I,0,0,-0.702779,False,4.420882,3.722633,1.334541,Sidney Jones
3,2018090600,190,(13:01) (Shotgun) M.Ryan pass deep left to J.J...,1,3,10,ATL,play_type_pass,PHI,39,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,5.0,"4 DL, 1 LB, 6 DB",SCRAMBLE_ROLLOUT_LEFT,0.0,0.0,13:01:00,49.0,NaN,NaN,C,33,33,3.047530,False,0.351283,0.864696,0.523927,Ronald Darby
4,2018090600,256,(10:59) (Shotgun) M.Ryan pass incomplete short...,1,3,1,ATL,play_type_pass,PHI,1,SHOTGUN,"2 RB, 3 TE, 0 WR",8.0,6.0,"6 DL, 3 LB, 2 DB",TRADITIONAL,0.0,0.0,10:59:00,11.0,NaN,NaN,I,0,0,-0.842272,False,1.287633,1.424114,1.800361,Malcolm Jenkins


## Create Dataframe with all defenders (at least 10 instances), the number of completions/incompletions, and their average distance from the receiver

In [54]:
valid_plays = play_info.loc[play_info['defender_receiver_dist'] >= 0]
defenders = valid_plays['closest_defender'].unique()
#print(len(defenders))
pass_defense = pd.DataFrame(columns=['display_Name', 'num_completions', 'num_incompletions', 'incomplete_percentage', 'average_dist'])
for defender in defenders:
    defender_plays = valid_plays.loc[valid_plays['closest_defender'] == defender]
    if len(defender_plays) < 20:
        continue
    num_completions = len(defender_plays.loc[defender_plays['passResult'] == 'C'])
    num_incompletions = len(defender_plays.loc[defender_plays['passResult'] != 'C'])
    average_dist = defender_plays['defender_receiver_dist'].mean()
    pass_defense = pass_defense.append({'display_Name': defender, 'num_completions': num_completions, 'num_incompletions': num_incompletions, 'incomplete_percentage': num_incompletions/(num_completions+num_incompletions), 'average_dist': average_dist}, ignore_index=True)
    #print(defender, num_completions, num_incompletions, average_dist)
print(pass_defense.shape)
pass_defense.head()


(182, 5)


,display_Name,num_completions,num_incompletions,incomplete_percentage,average_dist
0,Malcolm Jenkins,23,9,0.281250,3.486667
1,Jordan Hicks,35,13,0.270833,4.885250
2,Sidney Jones,22,10,0.312500,3.810169
3,Ronald Darby,36,23,0.389831,2.299893
4,Robert Alford,23,13,0.361111,2.074911


In [55]:
# top 10 defenders with highest incompletions %
pass_defense.sort_values(by=['incomplete_percentage'], ascending=False).head(20)

,display_Name,num_completions,num_incompletions,incomplete_percentage,average_dist
115,Eddie Jackson,9,14,0.608696,3.277061
64,Stephon Gilmore,17,26,0.604651,2.640089
165,Jimmie Ward,8,12,0.600000,4.019098
51,Eric Weddle,8,12,0.600000,3.129004
30,Jessie Bates,14,19,0.575758,3.972118
34,Xavien Howard,12,16,0.571429,4.203708
161,Richard Sherman,9,12,0.571429,4.120974
74,Barry Church,9,12,0.571429,3.884539
130,Rashaan Melvin,13,17,0.566667,2.083214
68,Jalen Ramsey,18,23,0.560976,2.454538


In [56]:
# top 10 defenders with least average distance

pass_defense.sort_values(by=['average_dist']).head(20)

,display_Name,num_completions,num_incompletions,incomplete_percentage,average_dist
28,William Jackson,19,16,0.457143,1.569671
106,Byron Jones,16,13,0.448276,1.733351
152,Trevor Williams,18,5,0.217391,1.753482
103,James Bradberry,25,17,0.404762,1.796879
53,Tony Jefferson,18,7,0.280000,1.827980
143,Trae Waynes,15,8,0.347826,1.840928
157,Coty Sensabaugh,11,10,0.476190,1.844687
133,John Johnson,17,15,0.468750,1.916533
67,Eli Apple,25,11,0.305556,1.992604
181,Troy Hill,10,10,0.500000,2.022989
